<a href="https://colab.research.google.com/github/samvass/capstone-llm-news-detector/blob/multi-modal/Copy_of_multimodal_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertModel, BertTokenizer
import torch.nn as nn
import torch.optim as optim
import os
from PIL import Image
from tqdm import tqdm

## Building the model

In [3]:
class MultiModalModel(nn.Module):
    def __init__(self):
        super(MultiModalModel, self).__init__()
        # Load pre-trained models
        self.resnet = resnet18(pretrained=True)
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Remove the last layer from both models
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1]) # what is avgpool layer?
        self.bert = nn.Sequential(*list(self.bert.children())[:-1]) # Note: Direct modification of BERT like this may not work as intended

        # Define concatonated layers
        self.multi_modal_layers = nn.Sequential(
            nn.Linear(in_features=self.resnet[-1].in_features + self.bert[-1].in_features, out_features=512),  # do we want to take the out_features of the new models instead?
            nn.ReLU(), # do we need this layer?
            nn.Linear(512, 2)  # binary classifier 0 or 1?
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, image_inputs, text_inputs):
        # Process image input
        image_features = self.resnet(image_inputs)
        image_features = torch.flatten(image_features, 1)  # Flatten the features

        # Process text input
        text_features = self.bert(**text_inputs).last_hidden_state[:, 0, :]  # Get the [CLS] token's features

        # Concatenate features
        combined_features = torch.cat((image_features, text_features), dim=1)

        # Pass through additional layers
        output = self.multi_modal_layers(combined_features)

        output_binary = self.sigmoid(output)

        return output

## Load Data

In [34]:
class MultiModalDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        """
        Args:
            dataframe (DataFrame): DataFrame with image paths, text, and labels.
            image_dir (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.text_idx = dataframe.columns.get_loc('Text')
        self.title_idx = dataframe.columns.get_loc('Title')
        self.image_idx = dataframe.columns.get_loc('Image')
        self.label_idx = dataframe.columns.get_loc('Label')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        print(self.image_dir, self.dataframe.iloc[idx, self.image_idx])
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, self.image_idx])
        image = Image.open(img_name).convert('RGB')

        text = self.dataframe.iloc[idx, self.text_idx]
        label = self.dataframe.iloc[idx, self.label_idx]
        title = self.dataframe.iloc[idx, self.title_idx]

        if self.transform:
            image = self.transform(image)

        # TODO: how will we convert the title, text to numbers??
        return title, text, image, label

In [35]:
# load data
ai_data = pd.read_csv('./ai_dataset.csv')
real_data = pd.read_csv('./real_dataset.csv')[['Title', 'Text', 'Image', 'Label']]

combined_data = pd.concat([ai_data, real_data])

In [36]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = MultiModalDataset(dataframe=combined_data, image_dir='./images/', transform=transform)

# Split the dataset
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


## Training

In [ ]:
model = MultiModalModel() # load model

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for image, text, label in tqdm(train_loader, desc=f'Train: epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.to(device), text.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(image, text)
        loss = criterion(torch.squeeze(outputs), labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()

    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, text, labels in val_loader:
            inputs, labels = inputs.to(device), text.to(device), labels.to(device)
            outputs = model(image, text)
            loss = criterion(torch.squeeze(outputs), labels.float())
            val_loss += loss.item()

            predicted = torch.squeeze(torch.round(torch.sigmoid(outputs)))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')


AttributeError: 'AdaptiveAvgPool2d' object has no attribute 'in_features'